# Dask clusters

(the material is based on the notebook https://github.com/jrbourbeau/hacking-dask)

This notebook covers Dask's distributed clusters in detail.

## Cluster overview

In this section we'll discuss:

1. The different components which make up a Dask cluster
2. Survey different ways to launch a cluster

<img src="img/dask-cluster.svg" width="600">

### Components of a cluster

A Dask cluster is composed of three different types of objects:

1. **Scheduler**: A single, centralized scheduler process which responds to requests for computations, maintains relavant state about tasks and worker, and sends tasks to workers to be computed.
2. **Workers**: One or more worker processes which compute tasks and store/serve their results.
3. **Clients**: One or more client objects which are the user-facing entry point to interact with the cluster.

A couple of notes about workers:

- Each worker runs in its own Python process. Each worker Python process has its own `concurrent.futures.ThreadPoolExecutor` which is uses to compute tasks in parallel.
- There's actually a fourth cluster object which is often not discussed: the **Nanny**. By default Dask workers are launched and managed by a separate nanny process. This separate process allows workers to restart themselves if you want to use the `Client.restart` method, or to restart workers automatically if they get above a certain memory limit threshold.

#### Related Documentation

- [Cluster architecture](https://distributed.dask.org/en/latest/#architecture)
- [Journey of a task](https://distributed.dask.org/en/latest/journey.html)

## Deploying Dask clusters

Deploying a Dask cluster means launching scheduler, worker, and client processes and setting up the appropriate network connections so these processes can communicate with one another. Dask clusters can be lauched in a few different ways which we will discuss later.

### Manual setup

Launch a scheduler process using the `dask-scheduler` command line utility:

```terminal
$ dask-scheduler
2023-07-20 20:42:29,894 - distributed.scheduler - INFO - -----------------------------------------------
2023-07-20 20:42:30,432 - distributed.scheduler - INFO - State start
2023-07-20 20:42:30,438 - distributed.scheduler - INFO - -----------------------------------------------
2023-07-20 20:42:30,439 - distributed.scheduler - INFO - Clear task state
2023-07-20 20:42:30,440 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.150.50:8786
2023-07-20 20:42:30,440 - distributed.scheduler - INFO -   dashboard at:                     :8787
2023-07-20 20:42:31,810 - distributed.scheduler - INFO - Register worker <WorkerState 'tls://oksana-2eshadura-40cern-2ech.dask-worker.coffea-opendata.casa:8788', name: kubernetes-worker-50d255ef-ba72-4162-af66-23fdbf942bb3, status: undefined, memory: 0, processing: 0>
2023-07-20 20:42:31,813 - distributed.scheduler - INFO - Starting worker compute stream, tls://oksana-2eshadura-40cern-2ech.dask-worker.coffea-opendata.casa:8788
2023-07-20 20:42:31,813 - distributed.core - INFO - Starting established connection
```

and then launch several workers by using the `dask-worker` command and providing them the address of the scheduler they should connect to:

```terminal
$ dask-worker tcp://192.0.0.100:8786
2023-07-20 20:45:36,940 - distributed.worker - INFO -       Start worker at: tcp://192.168.150.50:38077
2023-07-20 20:45:36,941 - distributed.worker - INFO -          Listening to: tcp://192.168.150.50:38077
2023-07-20 20:45:36,941 - distributed.worker - INFO -          dashboard at:       192.168.150.50:34591
2023-07-20 20:45:36,941 - distributed.worker - INFO - Waiting to connect to:  tcp://192.168.150.50:8786
2023-07-20 20:45:36,941 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,941 - distributed.worker - INFO -               Threads:                        112
2023-07-20 20:45:36,941 - distributed.worker - INFO -                Memory:                   4.00 GiB
2023-07-20 20:45:36,941 - distributed.worker - INFO -       Local Directory: /home/cms-jovyan/dask-worker-space/worker-q_4j0zc5
2023-07-20 20:45:36,941 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,949 - distributed.worker - INFO -         Registered to:  tcp://192.168.150.50:8786
2023-07-20 20:45:36,949 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,949 - distributed.core - INFO - Starting established connection
                         
```

## Cluster managers 

Dask has the notion of cluster manager objects. Cluster managers offer a consistent interface for common activities like adding/removing workers to a cluster, retrieving logs, etc.

<img src="img/dask-cluster-manager.svg" width="600">

### Dask LocalCluster

LocalCluster creates a "cluster" of a scheduler and workers running on the local machine.

Creating a cluster object will create a Dask scheduler and a number of Dask workers. If no arguments are specified then it will autodetect the number of CPU cores your system has and the amount of memory and create workers to appropriately fill that. You can also specify these arguments yourself.

In [1]:
from dask.distributed import LocalCluster
# Launch a scheduler and 4 workers on my local machine
cluster = LocalCluster(n_workers=2, threads_per_worker=2)
cluster

LocalCluster(3465d1aa, 'tcp://127.0.0.1:38781', workers=2, threads=4, memory=4.00 GiB)

In [2]:
from dask.distributed import LocalCluster
LocalCluster??

Init signature:
LocalCluster(
    name=None,
    n_workers=None,
    threads_per_worker=None,
    processes=None,
    loop=None,
    start=None,
    host=None,
    ip=None,
    scheduler_port=0,
    silence_logs=30,
    dashboard_address=':8787',
    worker_dashboard_address=None,
    diagnostics_port=None,
    services=None,
    worker_services=None,
    service_kwargs=None,
    asynchronous=False,
    security=None,
    protocol=None,
    blocked_handlers=None,
    interface=None,
    worker_class=None,
    scheduler_kwargs=None,
    scheduler_sync_interval=1,
    **worker_kwargs,
)
Source:        
class LocalCluster(SpecCluster):
    """Create local Scheduler and Workers

    This creates a "cluster" of a scheduler and workers running on the local
    machine.

    Parameters
    ----------
    n_workers: int
        Number of workers to start
    memory_limit: str, float, int, or None, default "auto"
        Sets the memory limit *per worker*.

        Notes regarding argument data

In [5]:
# Scale up to 3 workers
cluster.scale(3)
cluster

LocalCluster(3465d1aa, 'tcp://127.0.0.1:38781', workers=3, threads=6, memory=6.00 GiB)

In [6]:
# Scale down to 2 workers
cluster.scale(2)

In [7]:
# Retrieve cluster logs
cluster.get_logs()

{'Cluster': '',
 'Scheduler': "2023-07-24 14:18:51,772 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:41033', name: 2, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690208331.7723262')\n2023-07-24 14:17:59,933 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:41033\n2023-07-24 14:17:59,932 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:41033', name: 2, status: closed, memory: 0, processing: 0>\n2023-07-24 14:17:31,046 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:35951\n2023-07-24 14:17:31,045 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:35951', name: 1, status: running, memory: 0, processing: 0>\n2023-07-24 14:17:31,043 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:41799\n2023-07-24 14:17:30,298 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:41799', name: 0, status: running, memory: 0, processing: 0>\n2023-07-24 14:17:25,004 - distributed.scheduler - INFO -   dashboard at:  http://127.0.0.1:8787/status\n2023-07-24 14:17:25,003 - distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:38781\n2023-07-24 14:17:24,995 - distributed.scheduler - INFO - State start",
 'tcp://127.0.0.1:35951': '2023-07-24 14:17:31,050 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:31,049 - distributed.worker - INFO -         Registered to:      tcp://127.0.0.1:38781\n2023-07-24 14:17:31,047 - distributed.worker - INFO - Starting Worker plugin shuffle\n2023-07-24 14:17:28,634 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:28,634 - distributed.worker - INFO -       Local Directory: /tmp/dask-scratch-space/worker-6b8f0489\n2023-07-24 14:17:28,634 - distributed.worker - INFO -                Memory:                   2.00 GiB\n2023-07-24 14:17:28,634 - distributed.worker - INFO -               Threads:                          2\n2023-07-24 14:17:28,634 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:28,634 - distributed.worker - INFO - Waiting to connect to:      tcp://127.0.0.1:38781\n2023-07-24 14:17:28,634 - distributed.worker - INFO -          dashboard at:            127.0.0.1:37057\n2023-07-24 14:17:28,634 - distributed.worker - INFO -           Worker name:                          1\n2023-07-24 14:17:28,634 - distributed.worker - INFO -          Listening to:      tcp://127.0.0.1:35951\n2023-07-24 14:17:28,634 - distributed.worker - INFO -       Start worker at:      tcp://127.0.0.1:35951',
 'tcp://127.0.0.1:41799': '2023-07-24 14:17:31,046 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:31,045 - distributed.worker - INFO -         Registered to:      tcp://127.0.0.1:38781\n2023-07-24 14:17:31,044 - distributed.worker - INFO - Starting Worker plugin shuffle\n2023-07-24 14:17:28,696 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:28,696 - distributed.worker - INFO -       Local Directory: /tmp/dask-scratch-space/worker-ev1o97j9\n2023-07-24 14:17:28,696 - distributed.worker - INFO -                Memory:                   2.00 GiB\n2023-07-24 14:17:28,696 - distributed.worker - INFO -               Threads:                          2\n2023-07-24 14:17:28,696 - distributed.worker - INFO - -------------------------------------------------\n2023-07-24 14:17:28,696 - distributed.worker - INFO - Waiting to connect to:      tcp://127.0.0.1:38781\n2023-07-24 14:17:28,696 - distributed.worker - INFO -          dashboard at:            127.0.0.1:44009\n2023-07-24 14:17:28,695 - distributed.worker - INFO -           Worker name:                          0\n2023-07-24 14:17:28,695 - distributed.worker - INFO -          Listening to:      tcp

In [8]:
# Shut down cluster
cluster.close()